In [1]:
import requests # 웹에 데이터를 요청하기 위해
from lxml import html # 파싱을 위한 lxml import


# 데이터를 불러올 url
url = 'https://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=109'

# url로 웹에 데이터를 요청해서 response를 받음
# response에는 response 코드 200과 데이터(content)가 들어있음
html_response = requests.get(url)

# 전달 받은 response중에 content를 불러와서 데이터를 저장
tree = html.fromstring(html_response.content)

In [2]:
# location 태그들을 모아서 list 형태로 저장

# locations list에 있는 객체마다 밑에는 city,date,data 태그가 있음
locations = tree.xpath('//location')

In [12]:
import pandas as pd # 데이터 조작을 위해
import datetime # 시간형식을 사용하기 위해

# 데이터를 저장할 빈 데이터 프레임을 만듦
df = pd.DataFrame(columns=['도시','날짜','날씨','최고','최저'])

# 모든 locations list 안에있는 location 태그를 돌면서
for location in locations:
    
    # location 태그 바로 밑에 있는 city의 text를 받아서 city 변수에 저장
    city = location.xpath('.//city/text()')[0]
    
    # location 밑에 있는 data 태그에는 날씨,최고,최저,시간 태그들이 달려있음 그래서..
    datas = location.xpath('.//data')
    
    # data태그들 밑에 있는 데이터를 저장하기위해 for loop
    for data in datas:
        
        # 시간 태그에 있는 데이터를 문자열로 변환해서 저장
        date = str(data.xpath('.//tmef/text()')[0])
        
        # 문자열을 datetime 형식으로 저장
        date = datetime.datetime.strptime(date,'%Y-%m-%d %H:%M')
        
        # 날씨 태그의 text를 저장
        wf = data.xpath('.//wf/text()')[0]
        
        # 최저 기온 태그의 text를 저장
        tmn = data.xpath('.//tmn/text()')[0]
        
        # 최고 기온 태그의 text를 저장
        tmx = data.xpath('.//tmx/text()')[0]
        
        # 데이터들을 딕셔너리 형태로 만들어서
        temp_dict = {
            '도시' : city,
            '날짜' : date,
            '날씨' : wf,
            '최고' : tmn,
            '최저' : tmx
        }
        
        # 빈 데이터프레임에 추가
        df = df.append(temp_dict, ignore_index=True)
    
display(df)

,도시,날짜,날씨,최고,최저
0,서울,2021-07-26 00:00:00,맑음,26,35
1,서울,2021-07-26 12:00:00,맑음,26,35
2,서울,2021-07-27 00:00:00,맑음,25,35
3,서울,2021-07-27 12:00:00,맑음,25,35
4,서울,2021-07-28 00:00:00,맑음,25,34
...,...,...,...,...,...
450,여주,2021-07-30 00:00:00,구름많음,22,32
451,여주,2021-07-30 12:00:00,구름많음,22,32
452,여주,2021-07-31 00:00:00,구름많음,23,33
453,여주,2021-08-01 00:00:00,구름많음,24,31


In [4]:

# 연도만 따로 뽑아서 새 컬럼으로 
df['날짜_연도'] = df['날짜'].dt.year

# 월만 따로 뽑아서 새 컬럼으로 
df['날짜_월'] = df['날짜'].dt.month

# 날짜만 따로 뽑아서 새 컬럼으로 
df['날짜_일'] = df['날짜'].dt.day

# 날짜열 삭제
df.drop(['날짜'],axis=1,inplace=True)

In [5]:
df

,도시,날씨,최고,최저,날짜_연도,날짜_월,날짜_일
0,서울,맑음,26,35,2021,7,26
1,서울,맑음,26,35,2021,7,26
2,서울,맑음,25,35,2021,7,27
3,서울,맑음,25,35,2021,7,27
4,서울,맑음,25,34,2021,7,28
...,...,...,...,...,...,...,...
450,여주,구름많음,22,32,2021,7,30
451,여주,구름많음,22,32,2021,7,30
452,여주,구름많음,23,33,2021,7,31
453,여주,구름많음,24,31,2021,8,1


In [6]:
# 도시 원핫인코딩
df_city= pd.get_dummies(df['도시'])

# 날씨 원핫인코딩
df_weather = pd.get_dummies(df['날씨'])

In [7]:

# 도시와 날짜 열 추가
df = df.assign(**df_city)
df = df.assign(**df_weather)

#df = df.drop(['도시','날씨'],axis=1)
df

,도시,날씨,최고,최저,날짜_연도,날짜_월,날짜_일,가평,강화,고양,...,의정부,이천,인천,파주,평택,포천,하남,화성,구름많음,맑음
0,서울,맑음,26,35,2021,7,26,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,서울,맑음,26,35,2021,7,26,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,서울,맑음,25,35,2021,7,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,서울,맑음,25,35,2021,7,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,서울,맑음,25,34,2021,7,28,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,여주,구름많음,22,32,2021,7,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
451,여주,구름많음,22,32,2021,7,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
452,여주,구름많음,23,33,2021,7,31,0,0,0,...,0,0,0,0,0,0,0,0,1,0
453,여주,구름많음,24,31,2021,8,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [8]:

# feature 데이터 저장
x_data = df.iloc[:,2:-1]

# target 데이터 저장
y_data = df.loc[:,'최저']


# 테스트 해볼 데이터 예시 3개
y_yeoju = df.iloc[454,2:-1]
y_seoul = df.iloc[0,2:-1]
y_godincheon = df.iloc[13,2:-1]

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge,Lasso,ElasticNet, LinearRegression
from sklearn.pipeline import Pipeline

# train, test 데이터나누기
x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,test_size=0.2)

# 데이터 정규화하고 elastic search 모델 적용하기 위한 pipeline
modelElastic = Pipeline([('scl', StandardScaler()), ('clf',ElasticNet()),])

# GridSearchCV를 위한 하이퍼 파리미터 딕셔너리
param_value = {'clf__alpha': [0.001, 0.01, 1, 2, 3, 4],
              'clf__l1_ratio' : [0.1, 0.5, 1]}

# 모델생성
gridSearch = GridSearchCV(modelElastic, param_grid = param_value, cv = 10, verbose = 1, scoring = 'r2')

# 모델 학습
gridSearch.fit(x_train, y_train)

# 가장 좋은 하이퍼 파라미터 출력
gridSearch.best_params_

Fitting 10 folds for each of 18 candidates, totalling 180 fits


{'clf__alpha': 0.001, 'clf__l1_ratio': 1}

In [10]:
# 최고 score 출력
gridSearch.best_score_

0.999999438847863

In [11]:
import numpy as np

# 테스트

y_predict_yeoju = gridSearch.predict(np.array(y_yeoju.values).reshape(1,-1))


y_predict_seoul = gridSearch.predict(np.array(y_seoul.values).reshape(1,-1))


y_predict_godincheon = gridSearch.predict(np.array(y_godincheon.values).reshape(1,-1))


print(f'여주 예측 {int(y_predict_yeoju[0])},서울 예측 {int(y_predict_seoul[0])}, 인천 예측 {int(y_predict_godincheon[0])}')

여주 예측 31,서울 예측 34, 인천 예측 32
